## Criar o Assistente

In [1]:

import openai
import os
from dotenv import find_dotenv, load_dotenv

# load the api key
_ = load_dotenv(find_dotenv())
client = openai.Client(api_key=os.getenv("API_KEY"))

#### Cria o vetor

In [2]:
vector_store = client.beta.vector_stores.create(name="Apostila ASIMOV")

#### Envia os arquivos para o assistente

In [3]:
files = ["files/Explorando a API da OpenAI.pdf", "files/Explorando o Universo das IAs com Hugging Face.pdf"]

file_streamms = [open(file, "rb") for file in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id= vector_store.id,
    files=file_streamms,
)

In [32]:
result = client.beta.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-DhXvoFNcszP55Nf7XjwHFU', created_at=1741730561, last_error=None, object='vector_store.file', status='completed', usage_bytes=184818, vector_store_id='vs_67d0b26e2764819195ace1e9bf2a235c', chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'), attributes={}), VectorStoreFile(id='file-N6tHSN5Segqyk8gd4M9Uag', created_at=1741730561, last_error=None, object='vector_store.file', status='completed', usage_bytes=192730, vector_store_id='vs_67d0b26e2764819195ace1e9bf2a235c', chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'), attributes={})], has_more=False, object='list', first_id='file-DhXvoFNcszP55Nf7XjwHFU', last_id='file-N6tHSN5Segqyk8gd4M9Uag')


In [35]:
print(file_batch.status)
print(file_batch.file_counts.total)

completed
2


#### criando o assistente file search

In [37]:
assistant = client.beta.assistants.create(
    name="Tutor ASIMOV",
    instructions="Você é um tutor de uma escola de programação. Você é ótimo para responser perguntas teóricas sobre a api da OPENAI \
          e sobre a utilização da biblioteca HuggingFace com Python. Você utiliza as apostilas dos cursos para basear suas respostas. \
            Caso você não encontre as respostas nas apostilas informadas, você fala que não sabe responser.",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    model="gpt-4o-mini"
)

### Criando uma Thread

In [38]:
thread = client.beta.threads.create()

In [105]:
question = "Segundo o documento fornecido, o que é o Huggin face?"
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question
)

### Rodando a thread

In [106]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="O nome do usuário é Wellington e ele é um usuário Premium."
)

### Aguardart a thread rodar

In [107]:
import time
# roda várias vezes para identificar o status
while run.status in  ["queued", "in_progress", "cancelling", "failed" ]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run.status)

in_progress
in_progress
in_progress
in_progress
in_progress
completed


### Quando terminar de rodar

In [108]:
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages)
else:
    print(run.error)

SyncCursorPage[Message](data=[Message(id='msg_q99kfB5ViEr5Vy2UVTTAPEOD', assistant_id='asst_XMHT8QPcn1KMYAXCqgQtaolq', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=726, file_citation=FileCitation(file_id='file-N6tHSN5Segqyk8gd4M9Uag'), start_index=713, text='【16:1†source】', type='file_citation'), FileCitationAnnotation(end_index=1124, file_citation=FileCitation(file_id='file-N6tHSN5Segqyk8gd4M9Uag'), start_index=1111, text='【16:2†source】', type='file_citation'), FileCitationAnnotation(end_index=1303, file_citation=FileCitation(file_id='file-N6tHSN5Segqyk8gd4M9Uag'), start_index=1290, text='【16:5†source】', type='file_citation')], value='Conforme o documento, o Hugging Face é uma empresa que foi criada em 2017 na França, inicialmente focada no desenvolvimento de chatbots. Com o tempo, a empresa expandiu suas operações e começou a desenvolver uma infraestrutura própria, assim como bibliotecas em Python que simplificam

In [109]:
print(messages.data[0].content[0].text.value)

Conforme o documento, o Hugging Face é uma empresa que foi criada em 2017 na França, inicialmente focada no desenvolvimento de chatbots. Com o tempo, a empresa expandiu suas operações e começou a desenvolver uma infraestrutura própria, assim como bibliotecas em Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP). O Hugging Face se tornou uma plataforma colaborativa onde pesquisadores, empresas e entusiastas podem compartilhar modelos de inteligência artificial e conjuntos de dados para uma ampla gama de aplicações. Em 2023, a empresa atingiu um valor de mercado de 4,5 bilhões de dólares, tendo recebido investimentos de grandes empresas como Google, Meta, Microsoft e Nvidia【16:1†source】.

Além disso, a plataforma do Hugging Face é dividida em três áreas principais:
1. **Modelos**: Onde é possível buscar e visualizar todos os modelos de IA disponíveis.
2. **Spaces**: Aplicativos web que utilizam modelos de IA, desenvolvidos pela comunidade.
3. **Datasets**

### Analisando os passos do modelo

In [110]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [111]:
run_steps.data[0].step_details

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_q99kfB5ViEr5Vy2UVTTAPEOD'), type='message_creation')

In [112]:
for step in run_steps.data[::-1]:
    print("#step:", step.step_details.type)
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == "code_interpreter":
                print("> tool_call type:", tool_call.type)
                print("> input:")
                print(tool_call.code_interpreter.input)
                print("> outputs:", tool_call.code_interpreter.outputs)
            else:
                print("> tool_call type:", tool_call.type)
                print("> input:")
                print(tool_call)
        
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id, message_id=step.step_details.message_creation.message_id
        )
        print("--- message_text:", message.content[0].text.value)


#step: tool_calls
> tool_call type: file_search
> input:
FileSearchToolCall(id='call_SQ0CSmS9s0trqJeq2rlu8bW6', file_search=FileSearch(ranking_options=FileSearchRankingOptions(ranker='default_2024_08_21', score_threshold=0.0), results=[FileSearchResult(file_id='file-N6tHSN5Segqyk8gd4M9Uag', file_name='Explorando o Universo das IAs com Hugging Face.pdf', score=0.6833682153670512, content=None, attributes={}), FileSearchResult(file_id='file-N6tHSN5Segqyk8gd4M9Uag', file_name='Explorando o Universo das IAs com Hugging Face.pdf', score=0.6464814037049909, content=None, attributes={}), FileSearchResult(file_id='file-N6tHSN5Segqyk8gd4M9Uag', file_name='Explorando o Universo das IAs com Hugging Face.pdf', score=0.6216119790300367, content=None, attributes={}), FileSearchResult(file_id='file-N6tHSN5Segqyk8gd4M9Uag', file_name='Explorando o Universo das IAs com Hugging Face.pdf', score=0.48571616012481766, content=None, attributes={}), FileSearchResult(file_id='file-N6tHSN5Segqyk8gd4M9Uag', fil

In [113]:
nessages = client.beta.threads.messages.list(thread_id=thread.id)
message = list(messages)[0].content[0].text
annotations = message.annotations
quotes = []	
for index, annotation in enumerate(annotations):
        message.value = message.value.replace(annotation.text, f'[{index}]')
        if file_cit := getattr(annotation, 'file_citation', None):
            file = client.files.retrieve(file_cit.file_id)
            quotes.append(f'[{index}]: {file.filename}')
quotes = "\n".join(quotes)
message.value = f'{message.value}\n{quotes}'


In [114]:
print(message.value)

Conforme o documento, o Hugging Face é uma empresa que foi criada em 2017 na França, inicialmente focada no desenvolvimento de chatbots. Com o tempo, a empresa expandiu suas operações e começou a desenvolver uma infraestrutura própria, assim como bibliotecas em Python que simplificam o uso de modelos de Processamento de Linguagem Natural (NLP). O Hugging Face se tornou uma plataforma colaborativa onde pesquisadores, empresas e entusiastas podem compartilhar modelos de inteligência artificial e conjuntos de dados para uma ampla gama de aplicações. Em 2023, a empresa atingiu um valor de mercado de 4,5 bilhões de dólares, tendo recebido investimentos de grandes empresas como Google, Meta, Microsoft e Nvidia[0].

Além disso, a plataforma do Hugging Face é dividida em três áreas principais:
1. **Modelos**: Onde é possível buscar e visualizar todos os modelos de IA disponíveis.
2. **Spaces**: Aplicativos web que utilizam modelos de IA, desenvolvidos pela comunidade.
3. **Datasets**: Conjunto